In [67]:
import pandas as pd
import numpy as np

In [68]:
df = pd.read_csv("dataset.txt", sep="	")
df.tail()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
1710,Woodrow_Wilson,Was Wilson president of the American Political...,Yes,NaN,easy,S08_set3_a8
1711,Woodrow_Wilson,Did he not cast his ballot for John M. Palmer ...,Yes,NaN,easy,S08_set3_a8
1712,Woodrow_Wilson,Did Wilson not spend 1914 through the beginnin...,Yes,NaN,easy,S08_set3_a8
1713,Woodrow_Wilson,"Was Wilson , a staunch opponent of antisemitis...",Yes,NaN,easy,S08_set3_a8
1714,Woodrow_Wilson,What happened in 1917?,"raised billions through Liberty loans, imposed...",NaN,medium,S08_set3_a8


In [69]:
df.columns

Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')

In [70]:
df.drop("ArticleTitle",axis=1,inplace=True)

In [71]:
df.drop("Answer",axis=1,inplace=True)
df.drop("DifficultyFromAnswerer",axis=1,inplace=True)
df.drop("ArticleFile",axis=1,inplace=True)

In [72]:
df['DifficultyFromQuestioner'].fillna(df['DifficultyFromQuestioner'].mode()[0], inplace=True)

In [99]:
df['DifficultyFromQuestioner']=df['DifficultyFromQuestioner'].astype(str)
df['Question']=df['Question'].astype(str)

In [100]:
df.dtypes

Question                    object
DifficultyFromQuestioner    object
dtype: object

In [101]:
df

,Question,DifficultyFromQuestioner
0,Was Abraham Lincoln the sixteenth President of...,easy
1,Was Abraham Lincoln the sixteenth President of...,easy
2,Did Lincoln sign the National Banking Act of 1...,easy
3,Did Lincoln sign the National Banking Act of 1...,easy
4,Did his mother die of pneumonia?,easy
...,...,...
1710,Was Wilson president of the American Political...,easy
1711,Did he not cast his ballot for John M. Palmer ...,easy
1712,Did Wilson not spend 1914 through the beginnin...,easy
1713,"Was Wilson , a staunch opponent of antisemitis...",easy


In [102]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import LSTM,Activation,Dense,Dropout,Input,Embedding,SimpleRNN
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [103]:
x=df.Question
y=df.DifficultyFromQuestioner

In [104]:
le=LabelEncoder()
y=le.fit_transform(y)
y=y.reshape(-1,1)

In [105]:
y

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [106]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [107]:
max_words=1000
max_len=150
tok=Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences=tok.texts_to_sequences(X_train)
sequences_matrix=sequence.pad_sequences(sequences,maxlen=max_len)

In [108]:
def RNN():
    inputs=Input(name='inputs',shape=[max_len])
    layer=Embedding(max_words,50,input_length=max_len)(inputs)
    layer=LSTM(64)(layer)
    layer=Dense(256,name='FC1')(layer)
    layer=Activation('relu')(layer)
    layer=Dropout(0.5)(layer)
    layer=Dense(1,name='out_layer')(layer)
    layer=Activation('sigmoid')(layer)
    model=Model(inputs=inputs,outputs=layer)
    return model

In [109]:
model=RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 150)]             0         
                                                                 
 embedding (Embedding)       (None, 150, 50)           50000     
                                                                 
 lstm (LSTM)                 (None, 64)                29440     
                                                                 
 FC1 (Dense)                 (None, 256)               16640     
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 out_layer (Dense)           (None, 1)                 257   

In [110]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
9/9 [==============================] - 5s 305ms/step - loss: -0.6288 - accuracy: 0.4913 - val_loss: -5.6124 - val_accuracy: 0.5600
Epoch 2/10
9/9 [==============================] - 3s 384ms/step - loss: -10.3785 - accuracy: 0.5324 - val_loss: -16.3259 - val_accuracy: 0.5600
Epoch 3/10
9/9 [==============================] - 2s 245ms/step - loss: -21.0326 - accuracy: 0.5324 - val_loss: -26.5321 - val_accuracy: 0.5600
Epoch 4/10
9/9 [==============================] - 2s 252ms/step - loss: -31.6100 - accuracy: 0.5324 - val_loss: -37.1346 - val_accuracy: 0.5600
Epoch 5/10
9/9 [==============================] - 2s 248ms/step - loss: -42.4299 - accuracy: 0.5324 - val_loss: -48.1089 - val_accuracy: 0.5600
Epoch 6/10
9/9 [==============================] - 2s 249ms/step - loss: -54.2380 - accuracy: 0.5324 - val_loss: -59.5877 - val_accuracy: 0.5600
Epoch 7/10
9/9 [==============================] - 4s 437ms/step - loss: -66.6451 - accuracy: 0.5324 - val_loss: -72.0341 - val_accuracy: 0

In [111]:
test_sequences=tok.texts_to_sequences(X_test)
test_sequences_matrix=sequence.pad_sequences(test_sequences,maxlen=max_len)

In [112]:
acc=model.evaluate(test_sequences_matrix,Y_test)

11/11 [==============================] - 0s 36ms/step - loss: -123.1054 - accuracy: 0.5015


In [113]:
print('Test set\n Loss:{:0.4f}\n Accuracy: {:0.4f}'.format(acc[0],acc[1]))

Test set
 Loss:-123.1054
 Accuracy: 0.5015


In [119]:
test_text=input("Enter the text: ")
Testing_context=[test_text]
txts=tok.texts_to_sequences(Testing_context)
txts=sequence.pad_sequences(txts,maxlen=max_len)

Enter the text: Per rectum examination is not a useful test for diagnosis of


In [120]:
preds=model.predict(txts)
print(preds)

1/1 [==============================] - 0s 28ms/step
[[1.]]


In [121]:
if preds[0] > 0.66:
    print("hard")
elif preds[0]<0.66 and preds[0]>33.3:
    print("medium")
else:
    print("easy")

hard
